## Traditional machine learning
Feature(s): 'SMILES' column

- 'SMILES' column

Target: E_bin

- Binary class

In [18]:
import sys
import os

# Append the parent directory of your package to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))

In [19]:
import pandas as pd
import numpy as np
import zipfile

path_to_dataset = 'train_denis.csv'
csv_filename = 'train_denis.csv'

# Open the file, Correct the encoding and sep if necessary
if path_to_dataset.endswith('.zip'):
    with zipfile.ZipFile(path_to_dataset, 'r') as z:
        # Open the CSV file within the ZIP file
        with z.open(csv_filename) as f:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(f, sep=',', on_bad_lines='warn', index_col = 0)
else:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(path_to_dataset, sep=',', on_bad_lines='warn', index_col = 0)


print('Count of unique smiles:', df.SMILES.unique().shape[0])
print('Count of all of the smiles:', df.shape[0])


Count of unique smiles: 966
Count of all of the smiles: 966


In [20]:
display(df.head())
df.shape

,SMILES,Free energy of activation (kcal/mol),E_bin
0,CC(=O)c2nnc(c1ccc(OC)cc1)nn2,13.188984,0
1,c1ccccc1c2nnc(F)nn2,12.936726,0
2,c1cc(C(=O)OC)ccc1c2nnc(c1cnccc1)nn2,13.905599,0
3,c1cc(OC)ccc1c2nnc(c1ncccc1)nn2,14.327913,0
4,c1(NC)ccccc1c2nnc(CC)nn2,16.231148,1


(966, 3)

In [21]:
from MLPipeline import MLmodel, BinTheTarget

Target = ['E_bin']
Features = ['SMILES']
Feature_types = ['SMILES']
input = df

In [22]:
model = MLmodel(modelType='RandomForestClassifier',
                    df=input,
                    target=Target,
                    features=Features,
                    feature_types=Feature_types)

# get the values (input and output) of the model
X_train, X_test, y_train, y_test = model.getValues()

model.train()
model.evaluate()

2024-09-12 17:47:06.104 | INFO     | MLPipeline:__post_init__:134 - ndim y_train: 1
2024-09-12 17:47:06.105 | INFO     | MLPipeline:__post_init__:135 - ndim x_train: 2
2024-09-12 17:47:06.105 | INFO     | MLPipeline:__post_init__:136 - shape y_train: (50,)
2024-09-12 17:47:06.106 | INFO     | MLPipeline:__post_init__:137 - shape x_train: (50, 512)


RandomForestClassifier model trained successfully.
Accuracy for RandomForestClassifier: 0.72


0.72

In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.base import clone

def objective(trial, model_instance):
    """
    Objective function for Optuna to minimize.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_categorical('max_depth', [None, 10, 20, 30, 40]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 6),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # Clone the model to ensure a fresh instance each trial
    model_clone = clone(model_instance.model)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()

In [26]:
model = MLmodel(modelType='RandomForestClassifier', df=input, target=Target, 
                features=['SMILES'], hyperparameter_tuning=True,
                feature_types=Feature_types,
                optimization_method='optuna', objective=lambda trial: objective(trial, model))

model.train()
predictions = model.predict()
model.evaluate()

2024-09-12 17:47:51.290 | INFO     | MLPipeline:__post_init__:134 - ndim y_train: 1
2024-09-12 17:47:51.290 | INFO     | MLPipeline:__post_init__:135 - ndim x_train: 2
2024-09-12 17:47:51.291 | INFO     | MLPipeline:__post_init__:136 - shape y_train: (50,)
2024-09-12 17:47:51.291 | INFO     | MLPipeline:__post_init__:137 - shape x_train: (50, 512)
[I 2024-09-12 17:47:51,292] A new study created in memory with name: no-name-6291418d-53c6-4de6-8a2d-28250e59bf68
[I 2024-09-12 17:47:52,064] Trial 0 finished with value: 0.6 and parameters: {'n_estimators': 220, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 6, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 0.6.
[I 2024-09-12 17:47:53,505] Trial 1 finished with value: 0.6 and parameters: {'n_estimators': 295, 'max_depth': 30, 'min_samples_split': 13, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 0 with value: 0.6.
[I 2024-09-12 17:47:54,493] Trial 2 finished with valu

Best RandomForestClassifier model trained successfully with hyperparameter tuning using Optuna.
Best hyperparameters: {'n_estimators': 187, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False}
RandomForestClassifier model trained successfully.
Accuracy for RandomForestClassifier: 0.68


0.68